In [325]:
"""
Descripción del programa: El presente programa se trata de una implementación de una simulación en la que
uno o más robots de limpieza se encargan de limpiar las celdas sucias en un cuarto en un tiempo determinado.

Autor: Brenda Paola Castillo Torres (A01632227)

Última fecha de modificación: 16 de noviembre del 2021
"""
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

class CleaningModel(ap.Model):

    """
    Función setup: encargada de inicializar lo siguiente
    - Agentes involucrados (celdas y robots) -> cantidad, posición inicial y condición
    - Grid: simula el tamaño del cuarto cuyas celdas van a ser limpiadas
    """
    def setup(self):
        nCells = int(self.p['dirtyCellsDensity'] * (self.p.roomSize['X']*self.p.roomSize['Y'])) #density of dirty cells
        cells = self.dirty_cells = ap.AgentList(self, nCells)
        
        nRobots = self.p['robots']
        robots = self.cleaning_robots = ap.AgentList(self, nRobots)
        
        self.room = ap.Grid(self, [self.p['roomSize']['Y'], self.p['roomSize']['X']], track_empty=True) #room to be cleaned
        
        self.room.add_agents(cells, random=True, empty=True) #dirty cells in random positions
        
        positionsRobots = []
        for i in range(nRobots):
            positionsRobots.append((1, 1))
            
        self.room.add_agents(robots, positionsRobots) #start robots in position (1,1)
        self.dirty_cells.condition = 0      #0: drity cell
        self.cleaning_robots.condition = 2  #2: robot
    """
    Función step: en cada paso de tiempo, se encarga de limpiar celdas o mover a los agentes
    - Limpiar celdad: Si un robot se encuentra sobre una celda sucia, la limpia
    - Mover robot: Si un robot se encuentra en una celda limpia, se mueve a una celda adyacente aleatoria (vecino),
    si no puede moverse (porque ya hay otro robot en esa celda), se queda en el mismo lugar
    """    
    def step(self):
        robots = self.cleaning_robots.select(self.cleaning_robots.condition == 2)
        cells = self.dirty_cells.select(self.dirty_cells.condition == 0)
        
        posCells = []
        posRobots = []
        
        for cell in cells: #get current positions of dirty cells
            pos1 = self.room.positions[cell]
            posCells.append(pos1) 
            
        for robot in robots: #get current positions of robots
            pos1 = self.room.positions[robot]
            posRobots.append(pos1)
        
        for robot in robots:
            posRobot = self.room.positions[robot] #get current position of robot
            
            neighborList = self.room.neighbors(robot).to_list() #get list of neighbors
            newPosRobot = self.room.positions[self.random.choice(neighborList)] #choose a random position of neighbors
            
            if(self.room.attr_grid('condition')[posRobot] == 0 or posRobot in posCells): #if cell is dirty, clean
                for cell in cells:
                    posCell = self.room.positions[cell]
                    if(posCell == posRobot):
                        cell.condition = 1
                        posCells.remove(posRobot)
                        
            elif(self.room.attr_grid('condition')[newPosRobot] != 2 and newPosRobot not in posRobots): #move robot to random neighbor position
                posRobots.remove(posRobot)
                posRobots.append(newPosRobot)
                self.room.move_to(robot, newPosRobot) #move robot to random neighbor position, if possible
                robot.condition = 2
                self.p['moves'] += 1
                
            else: #if robot cannot move, stay in the same position
                self.room.move_to(robot, posRobot)
                robot.condition = 2
        
        if len(cells) == 0: #if all cells are clean, stop
            self.stop()
    
    """
    Función end: al finalizar la simulación, genera un reporte de
    - Porcentaje de celdas limpias al final
    - Total de movimientos realizados por los agentes (robots)
    """            
    def end(self):
        totalCells = int((self.p.roomSize['X']*self.p.roomSize['Y']))
        dirtyCellsLeft = len(self.dirty_cells.select(self.dirty_cells.condition == 0))
        cleanCells = totalCells - dirtyCellsLeft
        self.report('Percentage of clean cells:', cleanCells / totalCells * 100, " %")
        self.report("Total moves of agents:", self.p['moves'])
        
# Parámetros
parameters = {
    'dirtyCellsDensity': 0.8, 
    'robots': 10, #number of cleaning robots
    'roomSize': {'X': 10, 'Y': 10}, # Height and length of the grid
    'steps': 50, #maximum excecution time
    'moves': 0 #total moves of agents (final report)
}

"""
Función animation_plot: encargada de generar la visualización de la simulación, recibe el modelo y el eje
- Muestra el movimiento de los robots y cómo van limpiando celdas
- Muestra información de la simulación: tiempo transcurrido, Cantidad de celdas sucias, Porcentaje de celdas limpias, Movimiento total realizado por los robots
"""   
def animation_plot(model, ax):
    attr_grid = model.room.attr_grid('condition')
    color_dict = {0:'#ff0000', 1:'#008000', 2:'#000000', None:'#008000'} #0: dirty cells, 1: clean cells, 2: robots
    
    dirtyCellsLeft = len(model.dirty_cells.select(model.dirty_cells.condition == 0))
    totalCells = int((model.p.roomSize['X']*model.p.roomSize['Y']))
    cleanCells = totalCells - dirtyCellsLeft
    
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of cleaning cells\n"
                 f"Time-step: {model.t}, Dirty cells left: "
                 f"{dirtyCellsLeft}, "
                 f"Clean cells: {round(cleanCells / totalCells * 100,1)} %, "
                 f"Moves of agents: {model.p.moves}")

fig, ax = plt.subplots()
model = CleaningModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))